In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/lib/kaggle/gcp.py
/kaggle/input/scl-2021-ds/sampleSubmission.csv
/kaggle/input/scl-2021-ds/train.csv
/kaggle/input/scl-2021-ds/test.csv
/kaggle/working/__notebook_source__.ipynb


In [45]:
data = pd.read_csv('/kaggle/input/scl-2021-ds/train.csv')
data

,id,raw_address,POI/street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung
3,3,"toko dita, kertosono",toko dita/
4,4,jl. orde baru,/jl. orde baru
...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani
299996,299996,"raya cila kko, cilandak timur kel.",/raya cila kko
299997,299997,tanjung gusta jl. yaya 2 no 17,/
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/


In [46]:
data["POI"] = data["POI/street"].str.split(pat='/').apply(lambda x:x[0])
data["Street"] = data["POI/street"].str.split(pat='/').apply(lambda x:x[1])


In [47]:
data

,id,raw_address,POI/street,POI,Street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/,,
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung,,siung
3,3,"toko dita, kertosono",toko dita/,toko dita,
4,4,jl. orde baru,/jl. orde baru,,jl. orde baru
...,...,...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani,,jend ahmad yani
299996,299996,"raya cila kko, cilandak timur kel.",/raya cila kko,,raya cila kko
299997,299997,tanjung gusta jl. yaya 2 no 17,/,,
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/,taman asri,


In [62]:
TRAIN_DATA = []
STREET_DATA = set()
POI_DATA = set()

In [63]:
for index,row in data.iterrows():
    if index % 5000 == 0:
        print("Round ",index)
    entities = []
    if row["POI"] != '':
        start = 0
        end = 0
        find = row["raw_address"].find(row["POI"])
        if find == -1:
            sp = row["POI"].split(' ')
            count = 0
            for i in sp:
                v = i if len(i) < 3 else i[0:3]
                find = row["raw_address"].find(v)
                if find != -1:
                    if count == 0:
                        start = find
                    end = find
                    for j in range(len(i)):
                        if (find+j) < len(row["raw_address"]) and row["raw_address"][find+j] == i[j]:
                            end = find+j
                        else:
                            break
                    count+=1
        else:
            start = find
            end = find+len(row["POI"]) - 1
        POI_DATA.add(row["POI"])
        entities.append((start,end+1,row["POI"]))
    if row["Street"] != '':
        start = 0
        end = 0
        find = row["raw_address"].find(row["Street"])
        if find == -1:
            sp = row["Street"].split(' ')
            count = 0
            for i in sp:
                v = i if len(i) < 3 else i[0:3]
                find = row["raw_address"].find(v)
                if find != -1:
                    if count == 0:
                        start = find
                    end = find
                    for j in range(len(i)):
                        if (find+j) < len(row["raw_address"]) and row["raw_address"][find+j] == i[j]:
                            end = find+j
                        else:
                            break
                    count+=1
        else:
            start = find
            end = find+len(row["Street"]) - 1
        entities.append((start,end+1,row["Street"]))
        STREET_DATA.add(row["Street"])
    TRAIN_DATA.append((row["raw_address"],{'entities':entities}))

Round  0
Round  5000
Round  10000
Round  15000
Round  20000
Round  25000
Round  30000
Round  35000
Round  40000
Round  45000
Round  50000
Round  55000
Round  60000
Round  65000
Round  70000
Round  75000
Round  80000
Round  85000
Round  90000
Round  95000
Round  100000
Round  105000
Round  110000
Round  115000
Round  120000
Round  125000
Round  130000
Round  135000
Round  140000
Round  145000
Round  150000
Round  155000
Round  160000
Round  165000
Round  170000
Round  175000
Round  180000
Round  185000
Round  190000
Round  195000
Round  200000
Round  205000
Round  210000
Round  215000
Round  220000
Round  225000
Round  230000
Round  235000
Round  240000
Round  245000
Round  250000
Round  255000
Round  260000
Round  265000
Round  270000
Round  275000
Round  280000
Round  285000
Round  290000
Round  295000


In [64]:
TRAIN_DATA

[('jl kapuk timur delta sili iii lippo cika 11 a cicau cikarang pusat',
  {'entities': [(0, 40, 'jl kapuk timur delta sili iii lippo cika')]}),
 ('aye, jati sampurna', {'entities': []}),
 ('setu siung 119 rt 5 1 13880 cipayung', {'entities': [(5, 10, 'siung')]}),
 ('toko dita, kertosono', {'entities': [(0, 9, 'toko dita')]}),
 ('jl. orde baru', {'entities': [(0, 13, 'jl. orde baru')]}),
 ('raya samb gede, 299 toko bb kids',
  {'entities': [(20, 32, 'toko bb kids'), (0, 14, 'raya samb gede')]}),
 ('kem mel raya, no 4 bojong rawalumbu rt 1 36 rawalumbu',
  {'entities': [(0, 12, 'kem mel raya')]}),
 ('tela keuramat kuta alam', {'entities': [(0, 4, 'tela')]}),
 ('gg. i wates magersari', {'entities': [(0, 5, 'gg. i')]}),
 ('bunga ncole ix 2', {'entities': [(0, 14, 'bunga ncole ix')]}),
 ('cikahuripan sd neg boj 02 klap boj, no 5 16877',
  {'entities': [(12, 25, 'sd negeri bojong 02'), (26, 34, 'klap boj')]}),
 ('yaya atohar,', {'entities': [(0, 11, 'yayasan atohariyah')]}),
 ('abim ix 24 5'

In [65]:
POI_DATA

{'apotik sederhana',
 'smp bethani indonesia',
 'kaka azelea birdfarm',
 'mampang kel.',
 'pd shark tasikmalaya',
 'cafe maut',
 'toko aprilia',
 'taman eden 100',
 'taesin indonesia. pt',
 'panti al mubarokah',
 'istana sepeda',
 'meubel bp. rahmad',
 'warung makan dan angkringan mbak ilah',
 'arni',
 'selelah pasar baru',
 'pasar walasiho',
 'selfi laundry',
 'bara footwear',
 'sri sutami',
 'apotek sentral jaya',
 'masjid nur rohman',
 'koperasi pegawai negeri subur',
 'toko kue lapis',
 'toko malanggaten',
 'king ac',
 'metro motor ahass',
 'cahyo chicken farm',
 'pt. amalia surya cemerlang',
 'kios aulia',
 'medya salon',
 'fefi',
 'balai riset perikanan laut',
 'warung bakmi kang nardi',
 'caffe indomie',
 'penjahit taylor aktif',
 'cv. istamed group',
 'annida mini cafe',
 'sigit sucahyo',
 'madrasah tsanawiyah swasta al ishlahiyah',
 'watty craft',
 'depot yumna',
 'abadi diesel',
 'emping khalimin',
 'mushalla nurul istiqomah',
 'bocah modificator malang',
 'surya citra laundr

In [66]:
STREET_DATA

{'komp peru',
 'bage - cang',
 'dha hus indah sela vii',
 'raya pracima',
 'bint 15',
 'peru keni pono',
 'tid 6',
 'jl. neg',
 'jln raya kemantren',
 'gg. suka',
 'raya ampel dento',
 'singo iia',
 'dr maka 2',
 'ling barat 1',
 's p 2 marh',
 'salak v',
 'asa yudis',
 'kh. yusuf',
 'pengadegan timur 4',
 'kel ceng barat 11',
 'jl. cipinang baru 3, gg mel',
 'jl wisma tajur',
 'ko bumi mar emas timur iii',
 'gatot subroto barat',
 'cik viii',
 'komp mang per',
 'jati utara 2',
 'pelab tanjung emas',
 'saputan barat',
 'jln adonis samad',
 'pocut merah',
 'haji rean',
 'pen jalur 1',
 'aren jaya 2',
 'gg. sari asih',
 'raya kroya',
 'atang sand',
 'jl. sriwijaya',
 'tol cik - palimanan',
 'gang 7a',
 'lama v',
 'pipit iii',
 'bunga wijaya kes',
 'maru baru iv',
 'desa kalisuren',
 'bukit pakis utara 5',
 'duta inah malibu',
 'desa nogos',
 'gg. abim',
 'bumi ciman indah',
 'mayor s yafrie rach',
 'kap piere tand',
 'nam - koba',
 'mand sela 1',
 'komp.ha',
 'komp h muha',
 'mus nasir',

In [67]:
n_iter=20

In [68]:
import spacy
from tqdm import tqdm

In [69]:
ner_model = spacy.blank('en')
print("Create Blank en model")

Create Blank en model


In [70]:
if 'ner' not in ner_model.pipe_names:
    ner = ner_model.create_pipe('ner')
    ner_model.add_pipe(ner, last=True)
else:
    ner = ner_model.get_pipe('ner')

In [ ]:
for _,annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])
other_pipes = [pipe for pipe in ner_model.pipe_names if pipe != 'ner']

with ner_model.disable_pipes(*other_pipes):
    optimizer = ner_model.begin_training()
    for itn in range(n_iter):
        losses = {}
        for text,annotations in tqdm(TRAIN_DATA):
            ner_model.update([text],[annotations],drop=0.5,sgd=optimizer,losses=losses)
        print("Round",itn,losses)

In [ ]:
test_df = pd.read_csv('/kaggle/input/scl-2021-ds/test.csv')

In [ ]:
test_df

In [ ]:
TEST_RESULT = []

In [ ]:
for index,row in test_df.iterrows():
    doc = ner_model(row["raw_address"])
    poiTest = ''
    streetTest =''
    for ent in doc.ents:
        temp = ent.label_
        if temp in POI_DATA:
            poiTest = ent.text
        elif temp in STREET_DATA:
            streetTest = ent.text
    TEST_RESULT.append([row["id"],poiTest+'/'+streetTest])

In [ ]:
answerDF = pd.DataFrame(TEST_RESULT,columns=['id', 'POI/street'])

In [ ]:
answerDF.to_csv('/output/kaggle/working/output.csv')